Import the dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

Data collection and data processing

In [4]:
dataset = pd.read_csv("./data/sonar_data.csv", header=None)
# Generate column names
num_features = dataset.shape[1] - 1
column_names = [f'A{i+1}' for i in range(num_features)] + ['target']

# Set column names
dataset.columns = column_names
print(dataset.shape)
dataset.head()


(208, 61)


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A52,A53,A54,A55,A56,A57,A58,A59,A60,target
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [5]:
dataset.describe()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A51,A52,A53,A54,A55,A56,A57,A58,A59,A60
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [6]:
dataset['target'].value_counts()

target
M    111
R     97
Name: count, dtype: int64

More the data more accurate the model will be. Here, the data is not imbalanced because value counts of M and R is not that much different

In [7]:
# Mean value of the features for mines and rock
dataset.groupby('target').mean()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A51,A52,A53,A54,A55,A56,A57,A58,A59,A60
target,,,,,,,,,,,,,,,,,,,,,
M,0.034989,0.045544,0.050720,0.064768,0.086715,0.111864,0.128359,0.149832,0.213492,0.251022,...,0.019352,0.016014,0.011643,0.012185,0.009923,0.008914,0.007825,0.009060,0.008695,0.006930
R,0.022498,0.030303,0.035951,0.041447,0.062028,0.096224,0.114180,0.117596,0.137392,0.159325,...,0.012311,0.010453,0.009640,0.009518,0.008567,0.007430,0.007814,0.006677,0.007078,0.006024


In [8]:
# Separating data and labels
X = dataset.drop(columns='target', axis=1)
Y = dataset['target']

In [9]:
# Separate into train and test set
# Stratify = ensures that the proportion of each class in the target
# variable Y is preserved in both the training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.1, stratify=Y, random_state=2
)
print(Y_train.value_counts())
print(Y_test.value_counts())

target
M    100
R     87
Name: count, dtype: int64
target
M    11
R    10
Name: count, dtype: int64


Model training using Logistic Regression as this is binary class data

In [10]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [11]:
Y_pred = model.predict(X_test)
accuracy_score(Y_test,Y_pred)

0.9047619047619048

In [28]:
y_pred_proba = model.predict_proba(X_test)[::,1] 
y_pred_proba.shape
#y_pred_proba[0,0:]

(21,)

Using the model for single instance as a predictive system

In [13]:
input_data = (
    0.0270,
    0.0092,
    0.0145,
    0.0278,
    0.0412,
    0.0757,
    0.1026,
    0.1138,
    0.0794,
    0.1520,
    0.1675,
    0.1370,
    0.1361,
    0.1345,
    0.2144,
    0.5354,
    0.6830,
    0.5600,
    0.3093,
    0.3226,
    0.4430,
    0.5573,
    0.5782,
    0.6173,
    0.8132,
    0.9819,
    0.9823,
    0.9166,
    0.7423,
    0.7736,
    0.8473,
    0.7352,
    0.6671,
    0.6083,
    0.6239,
    0.5972,
    0.5715,
    0.5242,
    0.2924,
    0.1536,
    0.2003,
    0.2031,
    0.2207,
    0.1778,
    0.1353,
    0.1373,
    0.0749,
    0.0472,
    0.0325,
    0.0179,
    0.0045,
    0.0084,
    0.0010,
    0.0018,
    0.0068,
    0.0039,
    0.0120,
    0.0132,
    0.0070,
    0.0088,
)
# change the data to a numpy array
input_data_np = np.asarray(input_data)
input_data_reshaped = input_data_np.reshape(1, -1)

prediction = model.predict(input_data_reshaped)

if prediction == ["R"]:
    print("Rock")
else:
    print("Mine")

Rock


Explain the result using eli5 [explaing like i am 5]

In [14]:
import eli5
# from eli5.sklearn import explain_weights

# Explain the weights of the logistic regression model
#explanation = explain_weights(model, feature_names=column_names[:-1])
# eli5.show_weights(explanation)

eli5.show_weights(model)

ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (/home/shuvradeb/Machine Learning/ml-projects/.venv/lib/python3.8/site-packages/sklearn/utils/metaestimators.py)